# Introduction to Atmospheric Correction
### By Susan Meerdink<sup>1</sup>

<sup>1</sup>The Machine Learning and Sensing Lab, Electrical and Computer Engineering, University of Florida, Gainesville, FL 32611
<url>https://faculty.eng.ufl.edu/machine-learning/</url>

**Electromagnetic Spectrum Refresher**

* Term used to describe the entire range of light that exists. Generally for remote sensing imagery we work with the visible shortwave infrared (VSWIR) which ranges from 350 to 2500 nm. However, it can also apply to the thermal infrared (TIR) which ranges from 2500 nm to 15 um. Atmospheric correction can be applied to imagery measuring the VSWIR and TIR, but this introduction will focus on the VSWIR.
<img src="Picture01.png" alt="Electromagnetic Spectrum" style="width: 600px;"/>

**Remote Sensing Basics**

* Electromagnetic radiation is the dominant way evergy is transported from the sun to the earth. This energy passes from the sun through the atmosphere, where it is absorbed and reflected by particles/gasses/aerosols in the atmosphere. The energy then interacts with the earth's surface, also absorbing and reflecting biophysical and chemical properties of that surface. Energy is then reflected back through the atmosphere, where it is again influenced by atmopsheric components, to ultimately be measured by the sensor. 

NOTE: This is for passive remote sensing. Active remote sensing does not rely on the sun for energy, but generates its own energy source (e.g., lidar)
<img src="Picture02.png" alt="RS Basics" style="width: 600px;"/>

**Remote Sensing Data Pipeline**
In remote sensing, imagery can be delivered in multiple products. This terminology can vary across sensors, but for Landsat they are referred to as Levels so you would have Level 1B, Level 2, Level 3. Depending on your research question, you would need a different product level although for terrestrial applications reflectance products are the most common. Below is an explanation of that product pipeline.

* _Digitial Numbers (DNs)_: this data product is a quantized digital number of the amount of energy reflected from the surface and measured by a sensor. These values are sensor dependent and not useful for science because they are not physically meaning units. To convert from DNs to radiance, a gain and offset per band are applied to the data which are provided by the sensor designers or engineers. 
* _Radiance_: this data product is the amount of radiation (or energy) per pixel which has the units of µW/(cm^2*sr*nm). This data product includes artifacts/noise from the atmosphere due to scattering and absorption. 
* _Reflectance_: this data product is the proportion of total radiation from the sun striking a surface and is reflected back to the sensor. The units of this product is 0 - 100% reflectance since it is a proportion to the total amount of radiation available. This data product is created AFTER atmospheric correction to remove any artifacts/noise from the atmospheric components. 

Notes:
* The above is assuming that we are interested in the surface of the earth. There are radiance and reflectance products for the Top-of-atmosphere (TOA) which is the amount of radiation or reflectance measured by a space-based sensor flying higher than the earth's atmosphere. These products include contributions of the atmopshere including clouds, atmopsheric aerosols, and gases. This is often done to avoid performing atmospheric correction since it can be a complicated, time consuming process.

In [1]:
# imports and setup
import numpy as np
import os.path
import scipy.io
from loadmat import loadmat

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
default_dpi = mpl.rcParamsDefault['figure.dpi']
mpl.rcParams['figure.dpi'] = default_dpi*2 # run this to make figures larger, often have to execute block multiple times for it to take

ModuleNotFoundError: No module named 'loadmat'